# Echarts cheatsheet

## What is kandy-echarts

`kandy-echarts` is a wrapper around the [echarts library](https://echarts.apache.org/en/index.html). This library combines a clear, pure Kotlin API and elegant, beatiful charts from echarts.

## Installation

To get started in jupyter notebook with `kandy-echarts` just use [line magic](https://github.com/Kotlin/kotlin-jupyter#line-magics):

In [1]:
// use latest versions of library
%useLatestDescriptors
// kandy-echarts
%use kandy-echarts

## Data

`kandy` supports `Map` containing name as a key and list of elements as a value. Data requirements are matching list/column sizes and no `null` values.

In [2]:
val mapOfData = mapOf(
    "fruit" to listOf("apple", "orange", "grape"),
    "value" to listOf(12, 23, 44)
    )

To use the data, you need to create a typed pointer to the corresponding list with the same name as the key.

In [3]:
val fruit = column<String>("fruit")
val value = column<Int>("value")

`kandy` has [dataframe](https://kotlin.github.io/dataframe/overview.html) integration. Dataframe simplifies working with data and provides many operations on them.

To use it wtire the following line magic:

In [4]:
val df = dataFrameOf(
    "fruitDF" to listOf("apple", "orange", "grape"),
    "valueDF" to listOf(12, 23, 44)
    )

With a dataframe, there is no need to specifically create column references, since the dataframe provides us with type-safe access to columns using the column reference.

## Plot creation

To create a plot, use the `plot` function passing your data as an argument

In [5]:
plot(mapOfData){
    x(fruit)
    y(value)
    line {}
}

In the case of a dataframe, we need to call the `create` function on the data, this will allow us to use the columns of this dataframe. Then call the `plot` function inside the context.

In [7]:
df.plot { 
    x(fruitDF)
    y(valueDF)
    line {}
}

Otherwise, visualization of data from a `dataframe` and data from a `Map` do not differ.

## Layers, aesthetics, mappings and scales

To add the layears themselves, use the appropriate functions inside `plot`. So to add a line layer there is the `line` function, for a point layer - `points` and others. There are x and y for mapping data on axes.

In [8]:
val df = dataFrameOf(
    "name" to listOf("Alice", "Bob", "Charlie", "David"),
    "age" to listOf(17, 21, 15, 47),
    "height" to listOf(165, 183, 172, 169)
    )

In [9]:
df.plot { 
    x(name)
    line {
        y(age)
    }
}

In [10]:
df.plot { 
    points { 
        x(age)
        y(height)
    }
}

We have already met with `x` and `y`. These are *positional* aesthetic attributes. Each layer is characterized by its own set of aesthetic attributes, *aes* for short. Besides *positional* aes, there are also *non-positional* attributes (eg `color`, `size`, `width`). Such attributes are type safe. The aes value can be assigned in 2 ways: by setting and by mapping.

Setting is a simply setting constant value:

```kotlin
x(12.0f)
size(0.5)
color(Color.RED)
```

Mapping is a mapping from the data column to the values of the aesthetic attribute:

```kotlin
x(name)
size(age)
color(height)
```

The function of this mapping is called scale. Scales play a key role in data visualization. In the examples above, the mappings use the default scales, but we can specify the scales explicitly. There are two types of scales --- categorical (or discrete) and continuous, depends on its domain and range type. If scale is continuous, its domain and range are set using limits, while categorical scale domain and range are set like lists of categories and corresponding to them values. Also scales can be either positional or non-positional (depending on which aesthetic attributes are displayed). Refined scales (with explicit domain/range) are typed. Continuous scales have transform parameter; it defines the function type (which is linear by default). 

Here are the special functions for creating scales:
* `categorical()` - non-positional unspecified categorical scale
* `continuous()` - non-positional unspecified continuous scale
* `categoricalPos()` - positional unspecified categorical scale
* `continuousPos()` - positional unspecified continuous scale
* `categorical(listOf(true, false), listOf(Color.RED, Color.BLUE))` - non-positional categorical scale
* `continuous<Double, Double>(rangeLimits = 8.0 to 17.0)` - non-positional continuous scale
* `categoricalPos(listOf(1, 2, 4, 8, 16))` - positional categorical scale
* `continuousPos(0 to 260)` - positional continuous scale


To apply scale on column, simply use `.scaled()` extensin function of column and pass your scale as an argument. Note, that for refined scale its DomainType must match the type of column.

In [11]:
df.plot { 
    x(name)
    points { 
        y(height) { scale = continuous(150..200) }
        size(age) { scale = continuous(domain = 10..50, range = 15.0..55.0) }
    }
}

In [12]:
df.plot { 
    x(name)
    bars { 
        y(height) { scale = continuous(100..200) }
        color(age)
    }
}

## Layout

You can find all the settings for your chart in the `layout` layer. Such as plot *size*, *title*, *legend*, *grid* settings and others

In [13]:
df.plot { 
    layout { 
        legend { right = 0.px }
        tooltip { trigger = Trigger.AXIS }
        title.text = "Layout settings"
        textStyle.fontFamily = FontFamily.MONOSPACE
        size = 900 to 600
    }


    x(name)
    line { 
        name = "Line"
        y(height)
        color = Color.RED
        width = 5.0
    }
    bars { 
        name = "Bars"
        y(height)
        color = LinearGradient(0.0, 0.0, 0.0, 1.0, listOf(Color.GREY, Color.ORANGE))
    }
}